<a href="https://colab.research.google.com/github/XiaoxueRan/hku_phys3151_2022/blob/main/Assignment_5_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Initialize a network
# n_inputs: the number of inputs
# n_hidden: the number of neurons in hidden layer, each neuron has n_inputs+1 weights
# n_outputs: the number of outputs, each neuron has n_hidden+1 weights
def initialize_network(n_inputs, n_hidden1, n_hidden2, n_outputs):
    network = []
    hidden1_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden1)]
    network.append(hidden1_layer)
    hidden2_layer = [{'weights':[random() for i in range(n_hidden1 + 1)]} for i in range(n_hidden2)]
    network.append(hidden2_layer)
    output_layer = [{'weights':[random() for i in range(n_hidden2 +1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

In [ ]:
from random import seed
from random import random

seed(1)
network = initialize_network(8,3,5,2)

for layer in network:
    print(layer)

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614, 0.2550690257394217, 0.49543508709194095, 0.4494910647887381, 0.651592972722763, 0.7887233511355132, 0.0938595867742349]}, {'weights': [0.02834747652200631, 0.8357651039198697, 0.43276706790505337, 0.762280082457942, 0.0021060533511106927, 0.4453871940548014, 0.7215400323407826, 0.22876222127045265, 0.9452706955539223]}, {'weights': [0.9014274576114836, 0.030589983033553536, 0.0254458609934608, 0.5414124727934966, 0.9391491627785106, 0.38120423768821243, 0.21659939713061338, 0.4221165755827173, 0.029040787574867943]}]
[{'weights': [0.22169166627303505, 0.43788759365057206, 0.49581224138185065, 0.23308445025757263]}, {'weights': [0.2308665415409843, 0.2187810373376886, 0.4596034657377336, 0.28978161459048557]}, {'weights': [0.021489705265908876, 0.8375779756625729, 0.5564543226524334, 0.6422943629324456]}, {'weights': [0.1859062658947177, 0.9925434121760651, 0.8599465287952899, 0.12088995980580641]}, {'weights': [0

In [ ]:
from math import exp
# Calculate neuron activation for an input
def activate(weights, inputs):
    activation = weights[-1]
    for i in range(len(weights)-1):
        activation += weights[i] * inputs[i]
    return activation

# Transfer neuron activation
def transfer(activation):
    return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs)
            neuron['output'] = transfer(activation)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

# test forward propagation
network
row = [-0.5,-0.5,-0.1,-0.6,0.1,-0.1,0,-1.0]
output = forward_propagate(network, row)
print(output)

[0.9466571593328101, 0.9113223386037467]


In [ ]:
# Calculate the derivative of an neuron output
def transfer_derivative(output):
    return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = []
        if i != len(network)-1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])
            

In [ ]:
# Update network weights with error
def update_weights(network, row, l_rate):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

In [ ]:
# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagate(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            backward_propagate_error(network, expected)
            update_weights(network, row, l_rate)
        #print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
    print('The error is',sum_error)

In [ ]:
# import data
import pandas as pd
import numpy as np

df = pd.read_csv('support-vector-machine-example-1.csv')
dataset = df.values.tolist()

ave_var=[]
for j in range(len(dataset[0])-1):
    list=[dataset[ii][j] for ii in range(len(dataset))]
    ave_var.append([np.mean(list),np.sqrt(np.var(list))])
    
for i in range(len(dataset)):
    for j in range(len(dataset[0])-1):
        dataset[i][j]=(dataset[i][j]-ave_var[j][0])/ave_var[j][1]


seed(1)

n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 3,5, n_outputs)

train_network(network, dataset, 0.3, 3000, n_outputs)
for layer in network:
    print(layer)

The error is 199.05936454182915
[{'weights': [14.808159258295651, 23.216975996305226, -3.076975207473084, -8.36782894664908, -12.167110128794965, 6.673170133252304, 6.8167189487982025, 30.15850784676671, 9.786599898056009], 'output': 6.713050296434185e-23, 'delta': -2.917163325620383e-25}, {'weights': [-19.773193196042165, 21.41546370724876, -6.414362972053056, 1.5030457408047238, -19.026843304543178, 22.189071513569814, 8.841018328725255, 9.095798494396075, -3.7644103717585597], 'output': 0.0002073988569421751, 'delta': 6.355743773244441e-07}, {'weights': [15.48766045005111, 13.523684063464833, -6.004691669471638, -5.838894854113611, 17.676044570825738, 13.3761096806082, 7.727824394221108, -11.010947633229984, -13.990030311149425], 'output': 2.584855060718927e-23, 'delta': -5.726993092869951e-26}]
[{'weights': [13.25354166335216, 7.3909463259793124, 15.807318568542037, -25.75361600638043], 'output': 6.546548625459179e-12, 'delta': -4.037954054919169e-14}, {'weights': [-17.597140363713

In [ ]:
# Make a prediction with a network
def predict(network, row):
    outputs = forward_propagate(network, row)
    return outputs.index(max(outputs))

accuracy=0
for row in dataset:
    prediction = predict(network, row)
    print('Expected=%d, Got=%d' % (row[-1], prediction))
    if row[-1]==prediction:
        accuracy+=1.0/len(dataset)
        
print('The accuracy is', accuracy)

Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=0
Expected=1, Got=0
Expected=0, Got=1
Expected=1, Got=1
Expected=1, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=0
Expected=0, Got=0
Expected=1, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=1, Got=0
Expected=1, Got=1
Expected=1, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=1
Expected=1, Got=1
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=1
Expected=1, Got=0
Expected=1, Got=0
Expected=1, Got=1
Expected=0, Got=1
Expected=0, Got=1
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=1
Expected=1, Got=1
Expected=0, Got=1
Expected=0, Got=0
Expected=1, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=0, Got=0
Expected=1, Got=1
Expected=0, Got=1
Expected=0